## TASK LEFT
### 1.Checking Multiple percentages at which columns are dropped in 18th Cell
### 2.In Filling NaN's rather than -999 try (Mean,Mode,Median)

In [1]:
import numpy as np, modin.pandas as pd, os, gc, seaborn as sns
from sklearn import preprocessing
import warnings
warnings.simplefilter('ignore')
from sklearn.linear_model import LogisticRegression

In [2]:
os.listdir('../data/')

['test_transaction.csv',
 'train_identity.csv',
 'test_identity.csv',
 'sample_submission.csv',
 'train_transaction.csv']

In [3]:
%%time
train_identity = pd.read_csv("../data/train_identity.csv")
train_transaction = pd.read_csv("../data/train_transaction.csv")
test_transaction = pd.read_csv("../data/test_transaction.csv")
test_identity = pd.read_csv("../data/test_identity.csv")

CPU times: user 1.87 s, sys: 1.12 s, total: 3 s
Wall time: 1min 4s


In [4]:
print('Dimensions of the Train Identity set:',train_identity.shape)
print('Dimensions of the Train transaction set:',train_transaction.shape)
print('Dimensions of the Test transaction set:',test_transaction.shape)
print('Dimensions of the Test Identity set:',test_identity.shape)

Dimensions of the Train Identity set: (144233, 41)
Dimensions of the Train transaction set: (590540, 394)
Dimensions of the Test transaction set: (506691, 393)
Dimensions of the Test Identity set: (141907, 41)


In [5]:
train_identity.head()

,TransactionID,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987004,0.0,70787.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M
1,2987008,-5.0,98945.0,NaN,NaN,0.0,-5.0,NaN,NaN,NaN,...,mobile safari 11.0,32.0,1334x750,match_status:1,T,F,F,T,mobile,iOS Device
2,2987010,-5.0,191631.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,...,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,Windows
3,2987011,-5.0,221832.0,NaN,NaN,0.0,-6.0,NaN,NaN,NaN,...,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,NaN
4,2987016,0.0,7460.0,0.0,0.0,1.0,0.0,NaN,NaN,0.0,...,chrome 62.0,24.0,1280x800,match_status:2,T,F,T,T,desktop,MacOS


In [6]:
train_transaction.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
%%time
# Merge both the transaction and identity by left
train_data=pd.merge(train_transaction,train_identity,how="left",on="TransactionID")
test_data=pd.merge(test_transaction,test_identity,how="left",on="TransactionID")
dicti={}
for i in range(1,10):
    dicti.update({'id-0'+str(i):'id_0'+str(i)})
for i in range(10,39):
    dicti.update({'id-'+str(i):'id_'+str(i)})
test_data=test_data.rename(columns=dicti)
del dicti

CPU times: user 34.2 s, sys: 51.7 s, total: 1min 25s
Wall time: 2min 1s


In [8]:
# Print Shapes
print("Train Dataset shape: ", train_data.shape)
print("Test Dataset shape: ", test_data.shape)

Train Dataset shape:  (590540, 434)
Test Dataset shape:  (506691, 433)


In [9]:
train_data.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,...,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M


In [10]:
test_data.head()

,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,3663549,18403224,31.95,W,10409,111.0,150.0,visa,226.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3663550,18403263,49.00,W,4272,111.0,150.0,visa,226.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3663551,18403310,171.00,W,4476,574.0,150.0,visa,226.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3663552,18403310,284.95,W,10989,360.0,150.0,visa,166.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3663553,18403317,67.95,W,18018,452.0,150.0,mastercard,117.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
del train_identity
del train_transaction
del test_transaction
del test_identity

In [12]:
%%time
# Explore Categorical features
print('Training set:')
l1=[];l2=[];
for _ in train_data.columns:
    if train_data[_].dtypes == 'object' :
        value = len(train_data[_].unique())
        l1.append(_)
        l2.append(value)
frame=pd.DataFrame(np.column_stack((np.array(l1),np.array(l2))),columns=['Column_Name','Category_Count'])
frame

Training set:
CPU times: user 2.91 s, sys: 370 ms, total: 3.28 s
Wall time: 19.2 s


,Column_Name,Category_Count
0,ProductCD,5
1,card4,5
2,card6,5
3,P_emaildomain,60
4,R_emaildomain,61
5,M1,3
6,M2,3
7,M3,3
8,M4,4
9,M5,3


In [13]:
%%time 
print('Test set:')
l1=[];l2=[];
for _ in test_data.columns:
    if test_data[_].dtypes == 'object' :
        value = len(test_data[_].unique())
        l1.append(_)
        l2.append(value)
frame=pd.DataFrame(np.column_stack((np.array(l1),np.array(l2))),columns=['Column_Name','Category_Count'])
del l1,l2
frame

Test set:
CPU times: user 2.83 s, sys: 412 ms, total: 3.24 s
Wall time: 22.5 s


,Column_Name,Category_Count
0,ProductCD,5
1,card4,5
2,card6,4
3,P_emaildomain,61
4,R_emaildomain,61
5,M1,3
6,M2,3
7,M3,3
8,M4,4
9,M5,3


In [14]:
perc_fraud = len(train_data[train_data['isFraud']==1].index)*100/train_data.shape[0]
print("Percentage of Fradaulent records in dataset {:.2f}".format(perc_fraud) )

Percentage of Fradaulent records in dataset 3.50


In [15]:
# Check missing data - Many Columns have more than 50% NA/Null records
def missing_data(df) :
    count = df.isnull().sum()
    percent = (df.isnull().sum()) / (df.isnull().count()) * 100
    total = pd.concat([count, percent], axis=1, keys = ['Count', 'Percent'])
    types = []
    for col in df.columns :
        dtypes = str(df[col].dtype)
        types.append(dtypes)
    total['dtypes'] = types
    
    return np.transpose(total)

In [16]:
missing_data(train_data)

,C1,C10,C11,C12,C13,C14,C2,C3,C4,C5,...,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,isFraud
Count,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,512975,450258,512954,517251,512735,449555,449555,449555,449555,NaN
Percent,0,0,0,0,0,0,0,0,0,0,...,86.8654,76.2451,86.8619,87.5895,86.8248,76.1261,76.1261,76.1261,76.1261,0
dtypes,int64,int64,int64,float64,object,int64,float64,float64,object,float64,...,object,float64,object,object,object,object,object,object,object,object


In [17]:
missing_data(test_data)

,C1,C10,C11,C12,C13,C14,C2,C3,C4,C5,...,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38
Count,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,436032,370066,436020,436020,434516,369714,369714,369714,369714
Percent,0.000592077,0.000592077,0.000592077,0.000592077,0.93706,0.000592077,0.000592077,0.000592077,0.000592077,0.000592077,...,73.0056,86.0548,73.0358,86.0524,86.0524,85.7556,72.9664,72.9664,72.9664,72.9664
dtypes,int64,int64,float64,object,int64,float64,float64,object,float64,object,...,object,float64,object,object,object,object,object,object,object,object


In [18]:
percentage=70

In [19]:
#Here I have dropped columns who has NULL values more than given percentage
null_percent = train_data.isnull().sum()/train_data.shape[0]*100

cols_to_drop = np.array(null_percent[null_percent > percentage].index)

cols_to_drop

array(['dist2', 'R_emaildomain', 'D6', 'D7', 'D8', 'D9', 'D12', 'D13',
       'D14', 'V138', 'V139', 'V140', 'V141', 'V142', 'V143', 'V144',
       'V145', 'V146', 'V147', 'V148', 'V149', 'V150', 'V151', 'V152',
       'V153', 'V154', 'V155', 'V156', 'V157', 'V158', 'V159', 'V160',
       'V161', 'V162', 'V163', 'V164', 'V165', 'V166', 'V167', 'V168',
       'V169', 'V170', 'V171', 'V172', 'V173', 'V174', 'V175', 'V176',
       'V177', 'V178', 'V179', 'V180', 'V181', 'V182', 'V183', 'V184',
       'V185', 'V186', 'V187', 'V188', 'V189', 'V190', 'V191', 'V192',
       'V193', 'V194', 'V195', 'V196', 'V197', 'V198', 'V199', 'V200',
       'V201', 'V202', 'V203', 'V204', 'V205', 'V206', 'V207', 'V208',
       'V209', 'V210', 'V211', 'V212', 'V213', 'V214', 'V215', 'V216',
       'V217', 'V218', 'V219', 'V220', 'V221', 'V222', 'V223', 'V224',
       'V225', 'V226', 'V227', 'V228', 'V229', 'V230', 'V231', 'V232',
       'V233', 'V234', 'V235', 'V236', 'V237', 'V238', 'V239', 'V240',
       

In [20]:
# Dropping Columns
train_data = train_data.drop(cols_to_drop, axis=1)
test_data = test_data.drop(cols_to_drop,axis=1)

In [21]:
print(train_data.shape)
print(test_data.shape)

(590540, 226)
(506691, 225)


In [22]:
### Fill NaNs

train_data = train_data.fillna(-999)
test_data = test_data.fillna(-999)

In [23]:
train_y = train_data['isFraud']
train_X = train_data.drop('isFraud', axis=1)

In [24]:
# Label Encoding for categorical variables.
for _ in train_X.columns:
    if train_X[_].dtype=='object' or test_data[_].dtype=='object': 
        le = preprocessing.LabelEncoder()
        le.fit(list(train_X[_].values) + list(test_data[_].values))
        train_X[_] = le.transform(list(train_X[_].values))
        test_data[_] = le.transform(list(test_data[_].values))

RayTaskError: [36mray_worker[39m (pid=1272, ip=192.168.1.106)
  File "python/ray/_raylet.pyx", line 629, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 630, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 519, in ray._raylet.deserialize_args
ray.exceptions.RayTaskError: [36mray_worker[39m (pid=1277, ip=192.168.1.106)
  File "python/ray/_raylet.pyx", line 629, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 630, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 519, in ray._raylet.deserialize_args
ray.exceptions.UnreconstructableError: Object 9e941fd74985dfcbffffffff010000c001000000 is lost (either LRU evicted or deleted by user) and cannot be reconstructed. Try increasing the object store memory available with ray.init(object_store_memory=<bytes>) or setting object store limits with ray.remote(object_store_memory=<bytes>). See also: https://ray.readthedocs.io/en/latest/memory-management.html

In [ ]:
logreg = LogisticRegression()
logreg.fit(train_X, train_y)

In [ ]:
submission = pd.read_csv('../data/sample_submission.csv',index_col='TransactionID')
submission['isFraud'] = logreg.predict_proba(test_data)[:,1]
submission.to_csv('Logreg_submission.csv')
submission.head()